<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential()  # identity인 경우
        if stride != 1:  # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # (핵심) skip connection
        out = F.leaky_relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes  # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset

data_transformations = {
    "basic": transforms.Compose([
        transforms.ToTensor()
    ]),
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["crop_flip"])
augmented_train_data = ConcatDataset((train_dataset, train_aug_dataset))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 80:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1640625
Current benign train loss: 2.3052728176116943

Current batch: 100
Current benign train accuracy: 0.328125
Current benign train loss: 1.8430147171020508

Current batch: 200
Current benign train accuracy: 0.3671875
Current benign train loss: 1.734334945678711

Current batch: 300
Current benign train accuracy: 0.421875
Current benign train loss: 1.520750880241394

Current batch: 400
Current benign train accuracy: 0.484375
Current benign train loss: 1.4827640056610107

Current batch: 500
Current benign train accuracy: 0.484375
Current benign train loss: 1.4461308717727661

Current batch: 600
Current benign train accuracy: 0.578125
Current benign train loss: 1.2528612613677979

Current batch: 700
Current benign train accuracy: 0.453125
Current benign train loss: 1.419650912284851

Total benign train accuarcy: 42.591
Total benign train loss: 1237.757822394371

[ Test epoch: 0 ]

Test accuarcy: 26.31
Test average l


Test accuarcy: 81.6
Test average loss: 0.005921678161621094
Model Saved!

[ Train epoch: 8 ]

Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2634308636188507

Current batch: 100
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3888288736343384

Current batch: 200
Current benign train accuracy: 0.890625
Current benign train loss: 0.34935611486434937

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.22451278567314148

Current batch: 400
Current benign train accuracy: 0.8984375
Current benign train loss: 0.39780840277671814

Current batch: 500
Current benign train accuracy: 0.84375
Current benign train loss: 0.3794013261795044

Current batch: 600
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3429163694381714

Current batch: 700
Current benign train accuracy: 0.90625
Current benign train loss: 0.24719998240470886

Total benign train accuarcy: 90.782
Total benign train 


Total benign train accuarcy: 93.432
Total benign train loss: 149.37713573127985

[ Test epoch: 15 ]

Test accuarcy: 84.21
Test average loss: 0.005206216180324554
Model Saved!

[ Train epoch: 16 ]

Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.16652384400367737

Current batch: 100
Current benign train accuracy: 0.9296875
Current benign train loss: 0.22683468461036682

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.08469328284263611

Current batch: 300
Current benign train accuracy: 0.921875
Current benign train loss: 0.21902726590633392

Current batch: 400
Current benign train accuracy: 0.8984375
Current benign train loss: 0.25490057468414307

Current batch: 500
Current benign train accuracy: 0.921875
Current benign train loss: 0.1719389110803604

Current batch: 600
Current benign train accuracy: 0.9375
Current benign train loss: 0.17006702721118927

Current batch: 700
Current benign train accuracy: 0.91


Current batch: 700
Current benign train accuracy: 0.953125
Current benign train loss: 0.15635034441947937

Total benign train accuarcy: 94.37
Total benign train loss: 129.43402560427785

[ Test epoch: 23 ]

Test accuarcy: 87.91
Test average loss: 0.003972411504387856
Model Saved!

[ Train epoch: 24 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.16135352849960327

Current batch: 100
Current benign train accuracy: 0.9375
Current benign train loss: 0.2000439167022705

Current batch: 200
Current benign train accuracy: 0.96875
Current benign train loss: 0.10275653749704361

Current batch: 300
Current benign train accuracy: 0.921875
Current benign train loss: 0.19434179365634918

Current batch: 400
Current benign train accuracy: 0.953125
Current benign train loss: 0.1600143015384674

Current batch: 500
Current benign train accuracy: 0.9375
Current benign train loss: 0.19823595881462097

Current batch: 600
Current benign train accuracy: 0.96875
Curre


Current batch: 600
Current benign train accuracy: 0.9375
Current benign train loss: 0.15699173510074615

Current batch: 700
Current benign train accuracy: 0.96875
Current benign train loss: 0.10920237749814987

Total benign train accuarcy: 94.755
Total benign train loss: 120.50678850710392

[ Test epoch: 31 ]

Test accuarcy: 87.68
Test average loss: 0.004226348416507244
Model Saved!

[ Train epoch: 32 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.15010172128677368

Current batch: 100
Current benign train accuracy: 0.921875
Current benign train loss: 0.19153809547424316

Current batch: 200
Current benign train accuracy: 0.9609375
Current benign train loss: 0.12479102611541748

Current batch: 300
Current benign train accuracy: 0.96875
Current benign train loss: 0.11430562287569046

Current batch: 400
Current benign train accuracy: 0.9453125
Current benign train loss: 0.15041010081768036

Current batch: 500
Current benign train accuracy: 0.99218

...


Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.025271054357290268

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.00441290158778429

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.001396789331920445

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.00466273445636034

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0034997486509382725

Total benign train accuarcy: 99.799
Total benign train loss: 5.9702506408211775

[ Test epoch: 63 ]

Test accuarcy: 94.36
Test average loss: 0.0021128894317895175
Model Saved!

[ Train epoch: 64 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0014614867977797985

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03061910904943943

Current batch: 200
Current benign train accuracy: 1.0
Current benign tra


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0026383220683783293

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0049859401769936085

Current batch: 500
Current benign train accuracy: 0.984375
Current benign train loss: 0.024491097778081894

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.001497076591476798

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0017626399639993906

Total benign train accuarcy: 99.884
Total benign train loss: 3.8882529374968726

[ Test epoch: 71 ]

Test accuarcy: 94.66
Test average loss: 0.0020576435744762422
Model Saved!

[ Train epoch: 72 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0009294344345107675

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0008024250855669379

Current batch: 200
Current benign train accuracy: 1.0
Current benign tra


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.007636592723429203

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.004012206103652716

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.0096902409568429

Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.009970181621611118

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0038572668563574553

Total benign train accuarcy: 99.909
Total benign train loss: 3.0907033146941103

[ Test epoch: 79 ]

Test accuarcy: 94.3
Test average loss: 0.0021773895129561425
Model Saved!

[ Train epoch: 80 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0017687322106212378

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0007692023064009845

Current batch: 200
Current benign train accuracy: 1.0
Current benign t


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0005092616192996502

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.001081943861208856

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0011047114385291934

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.000932163093239069

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.001321642193943262

Total benign train accuarcy: 99.968
Total benign train loss: 1.5798343830683734

[ Test epoch: 87 ]

Test accuarcy: 94.85
Test average loss: 0.0020077582143247126
Model Saved!

[ Train epoch: 88 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0018721433589234948

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0007090105209499598

Current batch: 200
Current benign train accuracy: 1.0
Current benign train los


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0006335600628517568

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0008570238132961094

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0004948522546328604

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.003672318998724222

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.003376525128260255

Total benign train accuarcy: 99.976
Total benign train loss: 1.396245236042887

[ Test epoch: 95 ]

Test accuarcy: 94.97
Test average loss: 0.001961939420364797
Model Saved!

[ Train epoch: 96 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0008460191311314702

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0007130365702323616

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss